##Creazione del grafo
Importo dati e funzioni utili per lavorare con le comunità

In [ ]:
import google.colab.drive
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carica i dati
df = pd.read_csv('/content/drive/MyDrive/social_network/got_new/dati/Workspace_7.csv')

df_edges = pd.read_csv('/content/drive/MyDrive/social_network/got_new/dati/asoiaf-all-edges.csv')


###Funzioni utili per il plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_top_points_with_regression(df, x_attr, y_attr, num_points):
    """
    Plots a scatter plot of the top `num_points` data points with a regression line and error.

    Args:
        df: The pandas DataFrame containing the data.
        x_attr: The name of the column to use for the x-axis.
        y_attr: The name of the column to use for the y-axis.
        num_points: The number of top points to plot.
    """

    # Sort the DataFrame by x_attr in descending order
    sorted_df = df.sort_values(by=[x_attr], ascending=False)

    # Select the top num_points rows for plotting
    top_df = sorted_df.head(num_points)

    # Scatter plot for the top points with regression line and error
    sns.regplot(x=x_attr, y=y_attr, data=top_df, scatter_kws={'s': 10},
                line_kws={'color': 'red'}, truncate=False)
    plt.title(f'Relazione tra {x_attr} e {y_attr} (Top {num_points})')
    plt.xlabel(x_attr)
    plt.ylabel(y_attr)

    # Add labels (node IDs) to the top points
    for i in range(top_df.shape[0]):
        plt.text(x=top_df[x_attr].iloc[i], y=top_df[y_attr].iloc[i],
                 s=top_df['Id'].iloc[i], fontsize=5, color='gray')

    plt.show()

In [ ]:
def print_top_characters(df, attribute, num_characters):
    """
    Sorts the DataFrame by the given attribute and prints the top characters.

    Args:
        df: The pandas DataFrame containing the data.
        attribute: The name of the attribute to sort by (e.g., 'closnesscentrality').
        num_characters: The number of top characters to print.
    """

    # Sort the DataFrame by the given attribute in descending order
    sorted_df = df.sort_values(by=[attribute], ascending=False)

    # Select the top rows and the columns 'Id' and the given attribute
    top_characters = sorted_df[['Id', attribute]].head(num_characters)

    # Print the result
    print(top_characters)

In [ ]:
def plot_histogram(df, attribute):
    """
    Generates a histogram for the given attribute of a DataFrame.

    Args:
        df: The pandas DataFrame containing the data.
        attribute: The name of the attribute to create the histogram for.
    """
    sns.histplot(df[attribute], bins=15, kde=True, color='blue')
    plt.title(f'Distribuzione di {attribute}')  # Dynamic title using f-string
    plt.xlabel(attribute)
    plt.ylabel('Frequenza')
    plt.show()

In [ ]:
def plot_histogram_log_scale(df, attribute):
    """
    Generates a histogram with a logarithmic scale for the y-axis.

    Args:
        df: The pandas DataFrame containing the data.
        attribute: The name of the attribute to create the histogram for.
    """
    sns.histplot(df[attribute], bins=15, kde=True, color='blue', log_scale=(False, True))
    plt.title(f'Distribuzione di {attribute} (Scala Logaritmica)')
    plt.xlabel(attribute)
    plt.ylabel('Frequenza (Scala Logaritmica)')
    plt.show()

In [ ]:
def plot_graph(G, node_size=300, node_color='skyblue', edge_color='gray',
               with_labels=True, font_size=10, font_color='black',
               layout='spring', figsize=(8, 8)):

    # Imposta la figura e la dimensione
    plt.figure(figsize=figsize)

    pos = nx.spring_layout(G, seed=42)  # Layout a molla (force-directed)

    # Disegna i nodi
    nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=node_color)

    # Disegna gli archi
    nx.draw_networkx_edges(G, pos, edge_color=edge_color, alpha=0.7)

    # Disegna le etichette dei nodi (se richiesto)
    if with_labels:
        nx.draw_networkx_labels(G, pos, font_size=font_size, font_color=font_color)

    # Mostra il grafico
    plt.title("Network Graph")
    plt.axis('off')  # Disattiva gli assi
    plt.show()

###Calcolo statistiche di base per comunità

In [ ]:
import networkx as nx
import numpy as np

# Filtriamo i dati per modularity_class
modularity = np.unique(df['modularity_class'])

## The degree centrality values are normalized by dividing by the maximum
## possible degree in a simple graph n-1 where n is the number of nodes in G.


def degree_centrality_non_normalized(G):
    """Compute the non-normalized degree centrality for nodes.

    The non-normalized degree centrality for a node v is simply its degree.

    Parameters
    ----------
    G : graph
      A networkx graph

    Returns
    -------
    nodes : dictionary
       Dictionary of nodes with non-normalized degree centrality as the value.

    Examples
    --------
    >>> G = nx.Graph([(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)])
    >>> degree_centrality_non_normalized(G)
    {0: 3, 1: 3, 2: 2, 3: 2}
    """
    if len(G) == 0:  # Gestione del caso di grafo vuoto
        return {}

    centrality = {n: d for n, d in G.degree()}
    return centrality


# Funzione per calcolare le misure di centralità
def calculate_centrality_measures(group_df, group_edges):

  # Creiamo un grafo (graph) basato sulle informazioni contenute nel DataFrame
    G = nx.Graph()

    # Aggiungi i nodi
    for idx, row in group_df.iterrows():
        G.add_node(row['Id'], label=row['Label'], house=row['house'])

    # Aggiungi gli archi se le informazioni sono disponibili (ad esempio, grado o relazioni)
    # Aggiungi gli archi dal file degli archi (df_edges)
    for idx, row in group_edges.iterrows():
        source = row['Source']
        target = row['Target']
        G.add_edge(source, target)


    #print(f"Numero di nodi: {G.number_of_nodes()}")
    #print(f"Numero di archi: {G.number_of_edges()}")
    #plot_graph(G)

    # Misure di centralità
    centrality_measures = {}

    # Numero dei nodi
    centrality_measures['number_of_nodes'] = G.number_of_nodes()

    # Degree Centrality
    centrality_measures['degree_centrality'] = degree_centrality_non_normalized(G)

    # Betweenness Centrality
    centrality_measures['betweenness_centrality'] = nx.betweenness_centrality(G, normalized=True)

    # Closeness Centrality
    centrality_measures['closeness_centrality'] = nx.closeness_centrality(G)

    # PageRank
    centrality_measures['pagerank'] = nx.pagerank(G)

    # Coefficiente di clustering (clustering coefficient)
    centrality_measures['clustering_coefficient'] = nx.clustering(G)

    # Calcolo Eigen Vector
    centrality_measures['eigenvector_centrality'] = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)

    # Calcolo della densità all'interno della comunità
    num_internal_edges = G.number_of_edges()
    num_possible_edges = len(group_df) * (len(group_df) - 1) / 2
    centrality_measures['intra_density'] = num_internal_edges / num_possible_edges if num_possible_edges > 0 else 0


    return centrality_measures

Calcolo delle misure rispetto alle modularità

In [ ]:
# Calcola le misure per ciascun valore di 'modularity_class'
results = {}

for modularity_class in modularity:
    # Filtra il DataFrame per ciascun 'modularity_class'
    group_df = df[df['modularity_class'] == modularity_class]
    group_edges = df_edges[df_edges['Source'].isin(group_df['Id']) & df_edges['Target'].isin(group_df['Id'])]

    # Calcola le misure di centralità per il gruppo filtrato
    centrality_measures = calculate_centrality_measures(group_df, group_edges)

    # Salva i risultati per ogni classe
    results[modularity_class] = centrality_measures

In [ ]:
# Print ignorante
# Mostra i risultati
#for modularity_class, measures in results.items():
#    print(f"Modularity Class: {modularity_class}")
 #   for measure, values in measures.items():
 #       print(f"  {measure}: {values}")
 #   print()

Calcolo delle misure rispetto alle casate

In [ ]:
# Calcola le misure per ciascun valore di 'modularity_class'
resultsHouses = {}
houses = np.unique(df['house'])
houses = houses[houses != 'Other']

for house in houses:
    # Filtra il DataFrame per ciascun 'modularity_class'
    group_df = df[df['house'] == house]
    group_edges = df_edges[df_edges['Source'].isin(group_df['Id']) & df_edges['Target'].isin(group_df['Id'])]

    print("House", house)
    # Calcola le misure di centralità per il gruppo filtrato
    centrality_measures = calculate_centrality_measures(group_df, group_edges)

    # Salva i risultati per ogni classe
    resultsHouses[house] = centrality_measures

House Arryn
House Baratheon
House Bolton
House Cassel
House Clegane
House Frey
House Greyjoy
House Jon-Snow
House Lannister
House Martell
House Selmy
House Stark
House Targaryen
House Tarly
House Tully
House Tyrell


In [ ]:
# Print ignorante
# Mostra i risultati
for house, measures in sorted(resultsHouses.items(), key=lambda x: x[1]['number_of_nodes'], reverse=True):
    print(f"House: {house}")
    for measure, values in measures.items():
        print(f"  {measure}: {values}")
    print()

House: Frey
  number_of_nodes: 24
  degree_centrality: {'Aegon-Frey-(son-of-Stevron)': 2, 'Aenys-Frey': 1, 'Amerei-Frey': 0, 'Cleos-Frey': 0, 'Danwell-Frey': 3, 'Edwyn-Frey': 1, 'Elmar-Frey': 1, 'Emmon-Frey': 1, 'Hosteen-Frey': 3, 'Jared-Frey': 3, 'Lothar-Frey': 3, 'Merrett-Frey': 1, 'Olyvar-Frey': 1, 'Perwyn-Frey': 0, 'Petyr-Frey': 2, 'Rhaegar-Frey': 1, 'Roslin-Frey': 2, 'Ryman-Frey': 5, 'Stevron-Frey': 3, 'Tion-Frey': 0, 'Walda-Frey-(daughter-of-Merrett)': 0, 'Walder-Frey': 7, 'Walder-Frey-(son-of-Jammos)': 1, 'Walder-Frey-(son-of-Merrett)': 1}
  betweenness_centrality: {'Aegon-Frey-(son-of-Stevron)': 0.0, 'Aenys-Frey': 0.0, 'Amerei-Frey': 0.0, 'Cleos-Frey': 0.0, 'Danwell-Frey': 0.18972332015810275, 'Edwyn-Frey': 0.0, 'Elmar-Frey': 0.0, 'Emmon-Frey': 0.0, 'Hosteen-Frey': 0.05928853754940711, 'Jared-Frey': 0.05928853754940711, 'Lothar-Frey': 0.05928853754940711, 'Merrett-Frey': 0.0, 'Olyvar-Frey': 0.0, 'Perwyn-Frey': 0.0, 'Petyr-Frey': 0.07905138339920947, 'Rhaegar-Frey': 0.0, 'Roslin

# **ANALISI DELLE COMUNITA'**

##**Identificazione delle principali fazioni e alleanze**
####**Identificazione delle comunità principali**
L'obiettivo è determinare i gruppi principali o le fazioni presenti nella rete sociale di Game of Thrones. Questo è essenziale per comprendere come i personaggi si organizzano in sottoreti e per analizzare le dinamiche sociali tra gruppi come casate, alleanze e fazioni politiche.


Metriche usate:
* **Louvain Modularity**: Algoritmo utilizzato per rilevare le comunità massimizzando la modularità, che misura la densità delle connessioni all'interno delle comunità rispetto a quelle tra comunità diverse. Questo aiuta a identificare cluster di personaggi che interagiscono principalmente tra loro.
* **Inter- e intra-community density**: Misura il rapporto tra le connessioni all'interno di una comunità e quelle tra comunità diverse, utile per verificare la coesione di ogni gruppo.



###Sviluppo
Identificazione e calcolo misure di centralità delle comunità.

In [ ]:
# Calcola le misure per ciascun valore di 'modularity_class'
resultsComunities = {}

for modularity_class in modularity:
    # Filtra il DataFrame per ciascun 'modularity_class'
    group_df = df[df['modularity_class'] == modularity_class]

    rappresentante = group_df.loc[group_df['Authority'].idxmax()]
    auth_val = max(group_df['Authority'])

    group_edges = df_edges[df_edges['Source'].isin(group_df['Id']) & df_edges['Target'].isin(group_df['Id'])]

    print("Modularity", modularity_class, "- rappresentante:", rappresentante['Label'], "&", auth_val)
    # Calcola le misure di centralità per il gruppo filtrato
    centrality_measures = calculate_centrality_measures(group_df, group_edges)

    # Salva i risultati per ogni classe
    resultsComunities[modularity_class] = centrality_measures
    resultsComunities[modularity_class]['Authority'] = rappresentante['Label']

Modularity 0 - rappresentante: Arya Stark & 0.172624
Modularity 1 - rappresentante: Catelyn Stark & 0.175885
Modularity 2 - rappresentante: Stannis Baratheon & 0.182076
Modularity 3 - rappresentante: Unella & 0.009642
Modularity 4 - rappresentante: Alys Arryn & 0.00279
Modularity 5 - rappresentante: Gerold Hightower & 0.027488
Modularity 6 - rappresentante: Barristan Selmy & 0.098095
Modularity 7 - rappresentante: Balon Greyjoy & 0.043302
Modularity 8 - rappresentante: Horas Redwyne & 0.007511
Modularity 9 - rappresentante: Jaime Lannister & 0.226345
Modularity 10 - rappresentante: Tyrion Lannister & 0.251562
Modularity 11 - rappresentante: Jon Snow & 0.144182
Modularity 12 - rappresentante: Florian the Fool & 0.007212
Modularity 13 - rappresentante: Alla Tyrell & 0.012282
Modularity 14 - rappresentante: Alleras & 0.002925
Modularity 15 - rappresentante: Myrcella Baratheon & 0.07536
Modularity 16 - rappresentante: Jon Connington & 0.033735
Modularity 17 - rappresentante: Raynard & 0.00

####Calcolo della densità intra e inter comunitaria delle densità

In [ ]:
from collections import defaultdict

# Creiamo un grafo (graph) basato sulle informazioni contenute nel DataFrame
G = nx.Graph()

# Aggiungi i nodi
for idx, row in df.iterrows():
    G.add_node(row['Id'], label=row['Label'], house=row['house'], modularity=row['modularity_class'])

# Aggiungi gli archi se le informazioni sono disponibili (ad esempio, grado o relazioni)
# Aggiungi gli archi dal file degli archi (df_edges)
for idx, row in df_edges.iterrows():
    source = row['Source']
    target = row['Target']
    G.add_edge(source, target)

# Analizza connessioni inter-comunitarie
inter_connections = defaultdict(int)

for u, v in G.edges():
    comm_u = G.nodes[u]['modularity']
    comm_v = G.nodes[v]['modularity']
    if comm_u != comm_v:
      if(comm_u > comm_v):
        inter_connections[(comm_u, comm_v)] += 1
      else:
        inter_connections[(comm_v, comm_u)] += 1

# Analizza connessioni inter-comunitarie
    inter_connections2 = defaultdict(set)

    for u, v in G.edges():
        comm_u = G.nodes[u]['modularity']
        comm_v = G.nodes[v]['modularity']
        if comm_u != comm_v:
            inter_connections2[comm_u].add(comm_v)
            inter_connections2[comm_v].add(comm_u)

# Stampa i risultati
print("Densità intra-comunitaria:")
for modularity_class, measures in sorted(resultsComunities.items(), key=lambda x: x[1]['number_of_nodes'], reverse=False):
    print(f"Comunità {modularity_class} {measures['Authority']}: {measures['intra_density']:.2f} - {measures['number_of_nodes']} nodes")


print("\nConnessioni inter-comunitarie:")
for (comm_u, comm_v), count in sorted(inter_connections.items(), key=lambda x: x[1], reverse=True):
   print(f"Tra Comunità {comm_u} e {comm_v}: {count} collegamenti")


for comm_u, connected_communities in sorted(inter_connections2.items()):
        print(f"Comunità {comm_u}: {sorted(connected_communities)}")

Densità intra-comunitaria:
Comunità 4 Alys Arryn: 1.00 - 2 nodes
Comunità 8 Horas Redwyne: 1.00 - 2 nodes
Comunità 12 Florian the Fool: 1.00 - 2 nodes
Comunità 17 Raynard: 1.00 - 2 nodes
Comunità 3 Unella: 1.00 - 3 nodes
Comunità 13 Alla Tyrell: 1.00 - 3 nodes
Comunità 18 Waymar Royce: 1.00 - 3 nodes
Comunità 19 Willem Lannister: 0.67 - 3 nodes
Comunità 5 Gerold Hightower: 0.43 - 7 nodes
Comunità 14 Alleras: 0.35 - 11 nodes
Comunità 16 Jon Connington: 0.27 - 16 nodes
Comunità 15 Myrcella Baratheon: 0.21 - 23 nodes
Comunità 7 Balon Greyjoy: 0.08 - 44 nodes
Comunità 9 Jaime Lannister: 0.05 - 58 nodes
Comunità 2 Stannis Baratheon: 0.07 - 60 nodes
Comunità 0 Arya Stark: 0.09 - 63 nodes
Comunità 6 Barristan Selmy: 0.06 - 89 nodes
Comunità 11 Jon Snow: 0.05 - 114 nodes
Comunità 1 Catelyn Stark: 0.05 - 115 nodes
Comunità 10 Tyrion Lannister: 0.04 - 176 nodes

Connessioni inter-comunitarie:
Tra Comunità 10 e 1: 106 collegamenti
Tra Comunità 10 e 0: 96 collegamenti
Tra Comunità 10 e 9: 76 colle

Winner and losers nelle comunità

In [ ]:
# Trova le 5 casate con maggiore densità intra-comunitaria
top_5_density_comm = sorted(resultsComunities.items(), key=lambda x: x[1]['intra_density'], reverse=True)[:5]
##fai pesato rispetto al numero di nodi


# Trova le 5 coppie con il maggior numero di connessioni inter-comunitarie
top_5_connected_comm = sorted(inter_connections.items(), key=lambda x: x[1], reverse=True)[:5]

# Stampa i risultati
print("Top 5 casate con maggiore densità intra-comunitaria:")
for comm, measures in top_5_density_comm:
    print(f"Comunità {comm} {measures['Authority']}: {measures['intra_density']:.2f}")

print("\nTop 5 coppie di casate con il numero massimo di connessioni inter-comunitarie:")
for (comm_u, comm_v), count in top_5_connected_comm:
    print(f"Tra Comunità {comm_u} e {comm_v}: {count} collegamenti")


Top 5 casate con maggiore densità intra-comunitaria:
Comunità 3 Unella: 1.00
Comunità 4 Alys Arryn: 1.00
Comunità 8 Horas Redwyne: 1.00
Comunità 12 Florian the Fool: 1.00
Comunità 13 Alla Tyrell: 1.00

Top 5 coppie di casate con il numero massimo di connessioni inter-comunitarie:
Tra Comunità 10 e 1: 106 collegamenti
Tra Comunità 10 e 0: 96 collegamenti
Tra Comunità 10 e 9: 76 collegamenti
Tra Comunità 9 e 1: 48 collegamenti
Tra Comunità 10 e 2: 47 collegamenti


####Identificazione delle casate e calcolo delle metriche rispetto alle casate

In [ ]:
# Calcola le misure per ciascun valore delle casate
resultsHouses = {}
houses = np.unique(df['house'])
houses = houses[houses != 'Other']

for house in houses:
    # Filtra il DataFrame per ciascun 'house'
    group_df = df[df['house'] == house]
    group_edges = df_edges[df_edges['Source'].isin(group_df['Id']) & df_edges['Target'].isin(group_df['Id'])]

    # Calcola le misure di centralità per il gruppo filtrato
    centrality_measures = calculate_centrality_measures(group_df, group_edges)

    # Salva i risultati per ogni classe
    resultsHouses[house] = centrality_measures

####Calcolo misure della densità intra e iter casate.

In [ ]:
# Analizza connessioni inter-comunitarie
inter_house_connections = defaultdict(int)

# Creiamo un nuovo grafo
G_houses = nx.Graph()

# Ripulisco dai bastardi (inteso come nati fuori dal matrimonio)
df_no_Other = df[df['house'] != 'Other']
df_edges_no_Other = df_edges[df_edges['Source'].isin(df_no_Other['Id']) & df_edges['Target'].isin(df_no_Other['Id'])]

# Aggiungi i nodi
for idx, row in df_no_Other.iterrows():
    G_houses.add_node(row['Id'], label=row['Label'], house=row['house'], modularity=row['modularity_class'])

# Aggiungi gli archi
for idx, row in df_edges_no_Other.iterrows():
    source = row['Source']
    target = row['Target']
    G_houses.add_edge(source, target)

#plot_graph(G_houses)

# Calcolo la densità intra-comunitaria
for u, v in G_houses.edges():
    comm_u = G_houses.nodes[u]['house']
    comm_v = G_houses.nodes[v]['house']
    if comm_u != comm_v:
      if(comm_u > comm_v):
        inter_house_connections[(comm_u, comm_v)] += 1
      else:
        inter_house_connections[(comm_v, comm_u)] += 1


# Stampa i risultati
print("Densità intra-casata:")
for house, measures in sorted(resultsHouses.items(), key=lambda x: x[1]['number_of_nodes'], reverse=False):
    print(f"House {house} & {measures['intra_density']:.2f} & {measures['number_of_nodes']}")


print("\nConnessioni inter-casata:")
for (comm_u, comm_v), count in sorted(inter_house_connections.items(), key=lambda x: x[1], reverse=True):
    print(f"Tra casa {comm_u} e casa {comm_v}: {count} collegamenti")

Densità intra-casata:
House Jon-Snow & 0.00 & 1
House Selmy & 0.00 & 1
House Bolton & 1.00 & 2
House Clegane & 1.00 & 2
House Tarly & 1.00 & 2
House Cassel & 0.67 & 3
House Tully & 1.00 & 3
House Arryn & 0.40 & 5
House Baratheon & 0.64 & 8
House Greyjoy & 0.61 & 8
House Martell & 0.25 & 8
House Tyrell & 0.25 & 11
House Lannister & 0.33 & 12
House Stark & 0.50 & 12
House Targaryen & 0.10 & 21
House Frey & 0.08 & 24

Connessioni inter-casata:
Tra casa Stark e casa Lannister: 31 collegamenti
Tra casa Stark e casa Baratheon: 30 collegamenti
Tra casa Lannister e casa Baratheon: 27 collegamenti
Tra casa Stark e casa Frey: 24 collegamenti
Tra casa Targaryen e casa Lannister: 17 collegamenti
Tra casa Lannister e casa Frey: 17 collegamenti
Tra casa Tyrell e casa Lannister: 16 collegamenti
Tra casa Tyrell e casa Baratheon: 14 collegamenti
Tra casa Stark e casa Cassel: 12 collegamenti
Tra casa Targaryen e casa Baratheon: 11 collegamenti
Tra casa Tyrell e casa Stark: 11 collegamenti
Tra casa Stark

Riscalo le misure di densità sula base dei nodi, quindi rispetto al numero di persone presenti all'interno delle casate

In [ ]:
# Stampa i risultati ordinati
values = {}
print("Densità intra-casata normalizzati:")
for house, measures in resultsHouses.items():
    # Calcola il valore e lo aggiunge al dizionario
    value = measures['intra_density'] / measures['number_of_nodes']
    values[house] = value

# Ordina e stampa i risultati
for house, value in sorted(values.items(), key=lambda x: x[1], reverse=False):
    print(f"House {house}: {value:.4f}")


Densità intra-casata normalizzati:
House Jon-Snow: 0.0000
House Selmy: 0.0000
House Frey: 0.0032
House Targaryen: 0.0048
House Tyrell: 0.0231
House Lannister: 0.0278
House Martell: 0.0312
House Stark: 0.0417
House Greyjoy: 0.0759
House Arryn: 0.0800
House Baratheon: 0.0804
House Cassel: 0.2222
House Tully: 0.3333
House Bolton: 0.5000
House Clegane: 0.5000
House Tarly: 0.5000


####Analisi delle casate

Casata migliore

In [ ]:
# Trova la casata con maggiore densità intra-comunitaria
max_density_house = max(resultsHouses.items(), key=lambda x: x[1]['intra_density'])

# Trova le casate con il numero massimo di connessioni inter-comunitarie
max_connections = max(inter_house_connections.values())
most_connected_houses = [pair for pair, count in inter_house_connections.items() if count == max_connections]

# Stampa i risultati
print("Casata con maggiore densità intra-comunitaria:")
print(f"Casata {max_density_house[0]}: {max_density_house[1]['intra_density']:.2f}")

print("\nCoppie di casate con il numero massimo di connessioni inter-comunitarie:")
for comm_u, comm_v in most_connected_houses:
    print(f"Tra Comunità {comm_u} e {comm_v}: {max_connections} collegamenti")


Casata con maggiore densità intra-comunitaria:
Casata Bolton: 1.00

Coppie di casate con il numero massimo di connessioni inter-comunitarie:
Tra Comunità Stark e Lannister: 31 collegamenti


Casata peggiore

*La casata Greyjoy è connessa a tantissime casate, grazie a Theon che fa da bridge*

In [ ]:
# Trova la casata con minore densità intra-comunitaria
min_density_house = min(resultsHouses.items(), key=lambda x: x[1]['intra_density'])

# Trova le casate con il numero minimo di connessioni inter-comunitarie
min_connections = min(inter_house_connections.values())
least_connected_houses = [pair for pair, count in inter_house_connections.items() if count == min_connections]

# Stampa i risultati
print("Casata con minore densità intra-comunitaria:")
print(f"Casata {min_density_house[0]}: {min_density_house[1]['intra_density']:.2f}")

print("\nCoppie di casate con il numero minimo di connessioni inter-comunitarie:")
for comm_u, comm_v in least_connected_houses:
    print(f"Tra casa {comm_u} e casa {comm_v}: {min_connections} collegamenti")



Casata con minore densità intra-comunitaria:
Casata Jon-Snow: 0.00

Coppie di casate con il numero minimo di connessioni inter-comunitarie:
Tra casa Targaryen e casa Jon-Snow: 1 collegamenti
Tra casa Tarly e casa Targaryen: 1 collegamenti
Tra casa Selmy e casa Arryn: 1 collegamenti
Tra casa Selmy e casa Martell: 1 collegamenti
Tra casa Selmy e casa Clegane: 1 collegamenti
Tra casa Tully e casa Greyjoy: 1 collegamenti
Tra casa Tyrell e casa Targaryen: 1 collegamenti
Tra casa Targaryen e casa Greyjoy: 1 collegamenti
Tra casa Tarly e casa Jon-Snow: 1 collegamenti
Tra casa Jon-Snow e casa Greyjoy: 1 collegamenti
Tra casa Tarly e casa Lannister: 1 collegamenti
Tra casa Tyrell e casa Cassel: 1 collegamenti
Tra casa Tyrell e casa Martell: 1 collegamenti
Tra casa Stark e casa Martell: 1 collegamenti
Tra casa Frey e casa Cassel: 1 collegamenti


In [ ]:
# Trova le 5 casate con maggiore densità intra-comunitaria
top_5_density_houses = sorted(resultsHouses.items(), key=lambda x: x[1]['intra_density'], reverse=True)[:5]

# Trova le 5 coppie con il maggior numero di connessioni inter-comunitarie
top_5_connected_houses = sorted(inter_house_connections.items(), key=lambda x: x[1], reverse=True)[:5]

# Stampa i risultati
print("Top 5 casate con maggiore densità intra-casata:")
for house, measures in top_5_density_houses:
    print(f"Casata {house}: {measures['intra_density']:.2f}")

print("\nTop 5 coppie di casate con il numero massimo di connessioni inter-casata:")
for (comm_u, comm_v), count in top_5_connected_houses:
    print(f"Tra casa {comm_u} e casa {comm_v}: {count} collegamenti")

print()
print("Rimuoviamo le casate composte da 2 o 3 persone")
# Rimuoviamo outliers
houseFiltered = {} #defaultdict(int)
for h in houses:
  if h != 'Bolton' and h != 'Clegane' and h != 'Tarly' and h != 'Tully':
    houseFiltered[h] = resultsHouses[h]
top_5_density_houses = sorted(houseFiltered.items(), key=lambda x: x[1]['intra_density'], reverse=True)[:5]

# Stampa nuovi risultati
print("Top 5 casate con maggiore densità intra-casata:")
for house, measures in top_5_density_houses:
    print(f"Casata {house}: {measures['intra_density']:.2f}")

# Questa o cambia
#print("\nTop 5 coppie di casate con il numero massimo di connessioni inter-comunitarie:")
#for (comm_u, comm_v), count in top_5_connected_houses:
#    print(f"Tra Comunità {comm_u} e {comm_v}: {count} collegamenti")


Top 5 casate con maggiore densità intra-casata:
Casata Bolton: 1.00
Casata Clegane: 1.00
Casata Tarly: 1.00
Casata Tully: 1.00
Casata Cassel: 0.67

Top 5 coppie di casate con il numero massimo di connessioni inter-casata:
Tra casa Stark e casa Lannister: 31 collegamenti
Tra casa Stark e casa Baratheon: 30 collegamenti
Tra casa Lannister e casa Baratheon: 27 collegamenti
Tra casa Stark e casa Frey: 24 collegamenti
Tra casa Targaryen e casa Lannister: 17 collegamenti

Rimuoviamo le casate composte da 2 o 3 persone
Top 5 casate con maggiore densità intra-casata:
Casata Cassel: 0.67
Casata Baratheon: 0.64
Casata Greyjoy: 0.61
Casata Stark: 0.50
Casata Arryn: 0.40


In [ ]:
# Trova le 5 casate con minore densità intra-comunitaria
bottom_5_density_houses = sorted(resultsHouses.items(), key=lambda x: x[1]['intra_density'])[:5]

# Trova le 5 coppie con il minor numero di connessioni inter-comunitarie
bottom_5_connected_houses = sorted(inter_house_connections.items(), key=lambda x: x[1])[:5]

# Stampa i risultati
print("Top 5 casate con minore densità intra-casata:")
for house, measures in bottom_5_density_houses:
    print(f"Casata {house}: {measures['intra_density']:.2f}")

print("\nTop 5 coppie di casate con il numero minore di connessioni inter-casata:")
for (comm_u, comm_v), count in bottom_5_connected_houses:
    print(f"Tra casa {comm_u} e casa {comm_v}: {count} collegamenti")

Top 5 casate con minore densità intra-casata:
Casata Jon-Snow: 0.00
Casata Selmy: 0.00
Casata Frey: 0.08
Casata Targaryen: 0.10
Casata Martell: 0.25

Top 5 coppie di casate con il numero minore di connessioni inter-casata:
Tra casa Targaryen e casa Jon-Snow: 1 collegamenti
Tra casa Tarly e casa Targaryen: 1 collegamenti
Tra casa Selmy e casa Arryn: 1 collegamenti
Tra casa Selmy e casa Martell: 1 collegamenti
Tra casa Selmy e casa Clegane: 1 collegamenti


Winner e losers rispetto alla dimensione delle casate

In [ ]:
# nuovo set
filteredResults = {}

# filtro in base al numero dei nodi
for val in resultsHouses :
  if resultsHouses[val]['number_of_nodes'] < 4:
    print("Rimuovo casata", val)
  else:
    filteredResults[val] = resultsHouses[val]

# Filtro le casate
print()
print()
# Trova le 5 casate con maggiore densità intra-comunitaria
top_5_density_houses = sorted(filteredResults.items(), key=lambda x: x[1]['intra_density'], reverse=True)[:5]

# Trova le 5 coppie con il maggior numero di connessioni inter-comunitarie
top_5_connected_houses = sorted(inter_house_connections.items(), key=lambda x: x[1], reverse=True)[:5]

# Stampa i risultati
print("Top 5 casate con maggiore densità intra-comunitaria:")
for house, measures in top_5_density_houses:
    print(f"Casata {house}: {measures['intra_density']:.2f}")

print("\nTop 5 coppie di casate con il numero massimo di connessioni inter-comunitarie:")
for (comm_u, comm_v), count in top_5_connected_houses:
    print(f"Tra casa {comm_u} e casa {comm_v}: {count} collegamenti")

# Trova le 5 casate con minore densità intra-comunitaria
bottom_5_density_houses = sorted(filteredResults.items(), key=lambda x: x[1]['intra_density'])[:5]

# Trova le 5 coppie con il minor numero di connessioni inter-comunitarie
bottom_5_connected_houses = sorted(inter_house_connections.items(), key=lambda x: x[1])[:5]
print()
print()
# Stampa i risultati
print("Top 5 casate con minore densità intra-comunitaria:")
for house, measures in bottom_5_density_houses:
    print(f"Casata {house}: {measures['intra_density']:.2f}")

print("\nTop 5 coppie di casate con il numero minore di connessioni inter-comunitarie:")
for (comm_u, comm_v), count in bottom_5_connected_houses:
    print(f"Tra casa {comm_u} e casa {comm_v}: {count} collegamenti")

Rimuovo casata Bolton
Rimuovo casata Cassel
Rimuovo casata Clegane
Rimuovo casata Jon-Snow
Rimuovo casata Selmy
Rimuovo casata Tarly
Rimuovo casata Tully


Top 5 casate con maggiore densità intra-comunitaria:
Casata Baratheon: 0.64
Casata Greyjoy: 0.61
Casata Stark: 0.50
Casata Arryn: 0.40
Casata Lannister: 0.33

Top 5 coppie di casate con il numero massimo di connessioni inter-comunitarie:
Tra casa Stark e casa Lannister: 31 collegamenti
Tra casa Stark e casa Baratheon: 30 collegamenti
Tra casa Lannister e casa Baratheon: 27 collegamenti
Tra casa Stark e casa Frey: 24 collegamenti
Tra casa Targaryen e casa Lannister: 17 collegamenti


Top 5 casate con minore densità intra-comunitaria:
Casata Frey: 0.08
Casata Targaryen: 0.10
Casata Martell: 0.25
Casata Tyrell: 0.25
Casata Lannister: 0.33

Top 5 coppie di casate con il numero minore di connessioni inter-comunitarie:
Tra casa Targaryen e casa Jon-Snow: 1 collegamenti
Tra casa Tarly e casa Targaryen: 1 collegamenti
Tra casa Selmy e casa 

##**Determinazione del ruolo dei personaggi nelle comunità**

Chi sono i personaggi più influenti o centrali all'interno di ciascuna comunità? Questo è fondamentale per comprendere chi guida o coordina le attività e le dinamiche interne di un gruppo.


Metriche usate:
* **Degree centrality intra-comunità**: Misura i nodi con il maggior numero di connessioni all'interno della comunità, individuando i leader locali.
* **Betweenness centrality**: Individua i personaggi che collegano diverse comunità, facilitando l'interazione o la mediazione tra fazioni.

Degree identifica il numero di nodi a cui sono connesso, mentre l'authority è quante connessioni *importanti* ha il nodo.
Chiedi a mila:
- pagerank?
- ha senso rianalizzare i valori delle authority tagliando il dataset, o comunque analizzando solo un sottoinsieme dei dati (da gephi)


In [ ]:
# Dizionari per i risultati
leaders_by_degree = {}
mediators_by_betweenness = {}


# Calcola le metriche per ogni comunità
for community, nodes in resultsComunities.items():
    # Calcolo della degree centrality intra-comunità
    degree_centrality = resultsComunities[community]['degree_centrality']
    leader = max(degree_centrality.items(), key=lambda x: x[1])  # Nodo con massimo degree centrality
    leaders_by_degree[community] = leader

    # Calcolo della betweenness centrality (sull'intero grafo)
    betweenness_centrality = resultsComunities[community]['betweenness_centrality']
    if community == 3 or community == 13 or community == 18:
      print(community)
      print(betweenness_centrality)
    mediator = max(betweenness_centrality.items(), key=lambda x: x[1])  # Nodo con massimo betweenness centrality
    mediators_by_betweenness[community] = mediator



# Stampa dei risultati
print("Leader locali (degree centrality):")
for community, (node, centrality) in leaders_by_degree.items():
    print(f"Comunità {community} {node} & {centrality} ")

print("\nMediatori globali (betweenness centrality):")
for community, (node, centrality) in mediators_by_betweenness.items():
    print(f"Comunità {community} {node} & {centrality:.2f}")


3
{'Moelle': 0.0, 'Scolera': 0.0, 'Unella': 0.0}
13
{'Alla-Tyrell': 0.0, 'Elinor-Tyrell': 0.0, 'Megga-Tyrell': 0.0}
18
{'Gared': 0.0, 'Waymar-Royce': 0.0, 'Will-(prologue)': 0.0}
Leader locali (degree centrality):
Comunità 0 Arya-Stark & 50 
Comunità 1 Theon-Greyjoy & 47 
Comunità 2 Stannis-Baratheon & 31 
Comunità 3 Moelle & 2 
Comunità 4 Alys-Arryn & 1 
Comunità 5 Lewyn-Martell & 4 
Comunità 6 Daenerys-Targaryen & 58 
Comunità 7 Victarion-Greyjoy & 22 
Comunità 8 Hobber-Redwyne & 1 
Comunità 9 Jaime-Lannister & 34 
Comunità 10 Tyrion-Lannister & 75 
Comunità 11 Jon-Snow & 82 
Comunità 12 Florian-the-Fool & 1 
Comunità 13 Alla-Tyrell & 2 
Comunità 14 Pate-(novice) & 7 
Comunità 15 Arianne-Martell & 16 
Comunità 16 Aegon-Targaryen-(son-of-Rhaegar) & 9 
Comunità 17 Raynard & 1 
Comunità 18 Gared & 2 
Comunità 19 Willem-Lannister & 2 

Mediatori globali (betweenness centrality):
Comunità 0 Arya-Stark & 0.71
Comunità 1 Theon-Greyjoy & 0.45
Comunità 2 Stannis-Baratheon & 0.49
Comunità 3 Mo

In [ ]:
# Dizionari per i risultati
leaders_by_degree = {}
mediators_by_betweenness = {}


# Calcola le metriche per ogni comunità
for community, nodes in resultsHouses.items():
    # Calcolo della degree centrality intra-comunità
    degree_centrality = resultsHouses[community]['degree_centrality']
    leader = max(degree_centrality.items(), key=lambda x: x[1])  # Nodo con massimo degree centrality
    leaders_by_degree[community] = leader

    # Calcolo della betweenness centrality (sull'intero grafo)
    betweenness_centrality = resultsHouses[community]['betweenness_centrality']
    mediator = max(betweenness_centrality.items(), key=lambda x: x[1])  # Nodo con massimo betweenness centrality
    mediators_by_betweenness[community] = mediator

print(resultsHouses['Lannister'])

# Stampa dei risultati
print("Leader  (degree centrality):")
for community, (node, centrality) in leaders_by_degree.items():
    print(f"House {community} : Node {node} - {centrality:.2f} ")

print("\nGlobal mediators (betweenness centrality):")
for community, (node, centrality) in mediators_by_betweenness.items():
    print(f"House {community}: Node {node} - {centrality:.2f}")


{'number_of_nodes': 12, 'degree_centrality': {'Cersei-Lannister': 5, 'Daven-Lannister': 1, 'Genna-Lannister': 2, 'Jaime-Lannister': 8, 'Joanna-Lannister': 1, 'Kevan-Lannister': 6, 'Lancel-Lannister': 4, 'Martyn-Lannister': 2, 'Stafford-Lannister': 1, 'Tyrion-Lannister': 5, 'Tywin-Lannister': 7, 'Willem-Lannister': 2}, 'betweenness_centrality': {'Cersei-Lannister': 0.012727272727272726, 'Daven-Lannister': 0.0, 'Genna-Lannister': 0.0, 'Jaime-Lannister': 0.4096969696969697, 'Joanna-Lannister': 0.0, 'Kevan-Lannister': 0.15575757575757573, 'Lancel-Lannister': 0.0, 'Martyn-Lannister': 0.02424242424242424, 'Stafford-Lannister': 0.0, 'Tyrion-Lannister': 0.012727272727272726, 'Tywin-Lannister': 0.3539393939393939, 'Willem-Lannister': 0.012727272727272726}, 'closeness_centrality': {'Cersei-Lannister': 0.6470588235294118, 'Daven-Lannister': 0.4583333333333333, 'Genna-Lannister': 0.5238095238095238, 'Jaime-Lannister': 0.7857142857142857, 'Joanna-Lannister': 0.44, 'Kevan-Lannister': 0.6875, 'Lancel

*Unella e Moelle sono due suore (quelle che rasano Cersei)*

##**Esplorazione delle relazioni tra comunità**
Come interagiscono le diverse comunità tra loro? Identificare le relazioni tra fazioni permette di analizzare alleanze, conflitti e legami deboli o forti tra i gruppi principali.


Metriche usate:
* **Edge weight tra comunità**: Calcola la somma dei pesi degli archi tra membri di comunità diverse per analizzare la forza delle connessioni inter-gruppo.
* **Overlap tra nodi condivisi**: Misura il numero di personaggi che appartengono o interagiscono con più comunità, suggerendo possibili ponti tra gruppi rivali o alleati.

In [ ]:
from collections import defaultdict

def analyze_community_interactions(G):
    # Dizionario per somma dei pesi degli archi tra comunità
    inter_community_weights = defaultdict(float)

    # Dizionario per il conteggio di nodi che interagiscono con più comunità
    overlapping_nodes = defaultdict(set)

    # Itera sugli archi per calcolare i pesi tra comunità
    for u, v, data in G.edges(data=True):
        # Ottieni le comunità dei nodi u e v
        community_u = G.nodes[u]['modularity']
        community_v = G.nodes[v]['modularity']

        # Se appartengono a comunità diverse, somma il peso
        if community_u != community_v:
            weight = data.get('weight', 1)  # Usa il peso dell'arco, default 1
            if (community_u > community_v):
              inter_community_weights[(community_u, community_v)] += weight
            else:
              inter_community_weights[(community_v, community_u)] += weight


    # Identifica i nodi che interagiscono con più comunità
    for node in G.nodes():
        # Ottieni la comunità del nodo
        node_community = G.nodes[node]['modularity']
        # Cerca vicini appartenenti a comunità diverse
        connected_communities = set(G.nodes[neighbor]['modularity'] for neighbor in G.neighbors(node))
        if len(connected_communities) > 1:  # Se interagisce con più comunità
            overlapping_nodes[node] = connected_communities

    return inter_community_weights, overlapping_nodes

# G grafo complessivo
inter_weights, overlap_nodes = analyze_community_interactions(G)

# Stampa dei risultati
print("Pesi degli archi tra comunità:")
sorted_inter_weights = sorted(inter_weights.items(), key=lambda x: x[1], reverse=True)

for (comm_u, comm_v), weight in sorted_inter_weights:
    print(f"Tra Comunità {comm_u} e {comm_v}: peso totale {weight:.2f}")
    print(f"Comunità di {resultsComunities[comm_u]['Authority']} e di {resultsComunities[comm_v]['Authority']}")
    print("-")

Pesi degli archi tra comunità:
Tra Comunità 10 e 1: peso totale 106.00
Comunità di Tyrion Lannister e di Catelyn Stark
-
Tra Comunità 10 e 0: peso totale 96.00
Comunità di Tyrion Lannister e di Arya Stark
-
Tra Comunità 10 e 9: peso totale 76.00
Comunità di Tyrion Lannister e di Jaime Lannister
-
Tra Comunità 9 e 1: peso totale 48.00
Comunità di Jaime Lannister e di Catelyn Stark
-
Tra Comunità 10 e 2: peso totale 47.00
Comunità di Tyrion Lannister e di Stannis Baratheon
-
Tra Comunità 10 e 6: peso totale 45.00
Comunità di Tyrion Lannister e di Barristan Selmy
-
Tra Comunità 11 e 2: peso totale 40.00
Comunità di Jon Snow e di Stannis Baratheon
-
Tra Comunità 11 e 10: peso totale 30.00
Comunità di Jon Snow e di Tyrion Lannister
-
Tra Comunità 15 e 10: peso totale 28.00
Comunità di Myrcella Baratheon e di Tyrion Lannister
-
Tra Comunità 1 e 0: peso totale 25.00
Comunità di Catelyn Stark e di Arya Stark
-
Tra Comunità 11 e 1: peso totale 25.00
Comunità di Jon Snow e di Catelyn Stark
-
Tra

In [ ]:

print("\nNodi che interagiscono con più comunità:")
for node, communities in overlap_nodes.items():
    print(f"Nodo {node} interagisce con le comunità: ")
    for community in communities:  # Itera sulle singole comunità
        authority = resultsComunities[community]['Authority']  # Ottieni il valore di Authority
        print(f"  Comunità {community}: Authority {authority}")


Nodi che interagiscono con più comunità:
Nodo Addam-Marbrand interagisce con le comunità: 
  Comunità 1: Authority Catelyn Stark
  Comunità 10: Authority Tyrion Lannister
  Comunità 9: Authority Jaime Lannister
Nodo Aegon-I-Targaryen interagisce con le comunità: 
  Comunità 0: Authority Arya Stark
  Comunità 1: Authority Catelyn Stark
  Comunità 2: Authority Stannis Baratheon
  Comunità 6: Authority Barristan Selmy
  Comunità 10: Authority Tyrion Lannister
Nodo Aegon-Targaryen-(son-of-Rhaegar) interagisce con le comunità: 
  Comunità 16: Authority Jon Connington
  Comunità 10: Authority Tyrion Lannister
  Comunità 6: Authority Barristan Selmy
Nodo Aemon-Targaryen-(Maester-Aemon) interagisce con le comunità: 
  Comunità 2: Authority Stannis Baratheon
  Comunità 6: Authority Barristan Selmy
  Comunità 10: Authority Tyrion Lannister
  Comunità 11: Authority Jon Snow
  Comunità 14: Authority Alleras
Nodo Aenys-Frey interagisce con le comunità: 
  Comunità 0: Authority Arya Stark
  Comunit

Top 5

In [ ]:
# Top 5 per pesi degli archi tra comunità
top_5_inter_weights = sorted(inter_weights.items(), key=lambda x: x[1], reverse=True)[:5]

print("Top 5 pesi degli archi tra comunità:")
for (comm_u, comm_v), weight in top_5_inter_weights:
    print(f"Tra Comunità {comm_u} e {comm_v}: peso totale {weight:.2f}")
    print(f"Comunità di Authority {resultsComunities[comm_u]['Authority']} e di Authority {resultsComunities[comm_v]['Authority']}")

# Top 5 per nodi che interagiscono con più comunità
top_5_overlap_nodes = sorted(overlap_nodes.items(), key=lambda x: len(x[1]), reverse=True)[:5]

print("\nTop 5 nodi che interagiscono con più comunità:")
for node, communities in top_5_overlap_nodes:
    print(f"Nodo {node} interagisce con le comunità: {', '.join(map(str, communities))}")
    #for community in communities:  # Itera sulle singole comunità
    #    authority = resultsComunities[community]['Authority']  # Ottieni il valore di Authority
    #    print(f"  Comunità {community}: Authority {authority}")
    print("-")


Top 5 pesi degli archi tra comunità:
Tra Comunità 10 e 1: peso totale 106.00
Comunità di Authority Tyrion Lannister e di Authority Catelyn Stark
Tra Comunità 10 e 0: peso totale 96.00
Comunità di Authority Tyrion Lannister e di Authority Arya Stark
Tra Comunità 10 e 9: peso totale 76.00
Comunità di Authority Tyrion Lannister e di Authority Jaime Lannister
Tra Comunità 9 e 1: peso totale 48.00
Comunità di Authority Jaime Lannister e di Authority Catelyn Stark
Tra Comunità 10 e 2: peso totale 47.00
Comunità di Authority Tyrion Lannister e di Authority Stannis Baratheon

Top 5 nodi che interagiscono con più comunità:
Nodo Robert-Baratheon interagisce con le comunità: 0, 1, 2, 5, 6, 7, 9, 10, 11, 15, 16
-
Nodo Cersei-Lannister interagisce con le comunità: 0, 1, 2, 3, 6, 9, 10, 11, 15, 17
-
Nodo Sansa-Stark interagisce con le comunità: 0, 1, 2, 6, 9, 10, 11, 12, 13, 15
-
Nodo Tyrion-Lannister interagisce con le comunità: 0, 1, 2, 6, 7, 9, 10, 11, 15, 16
-
Nodo Eddard-Stark interagisce con l

##**Individuazione delle comunità isolate o periferiche**
Quali gruppi sono isolati dal resto della rete? L'analisi delle comunità periferiche è utile per comprendere quali fazioni o gruppi sono meno integrati nella trama principale o rappresentano elementi esterni alla narrazione centrale.


Metriche usate:
* **Modularità locale**: Misura il livello di separazione di una comunità rispetto al resto della rete. Un valore alto indica una comunità isolata.
* **Closeness inter-comunità**: Analizza la distanza media tra nodi periferici e il resto della rete per individuare gruppi meno accessibili.

In [ ]:
def find_isolated_groups(G):
    # Dizionario per modularità locale e closeness inter-comunità
    community_isolation = {}

    # Raggruppa i nodi per comunità
    communities = defaultdict(list)
    for node, data in G.nodes(data=True):
        communities[data['modularity']].append(node)

    for community, nodes in communities.items():
        # Sottografo della comunità
        subgraph = G.subgraph(nodes)

        # 1. Calcolo della modularità locale
        intra_edges = subgraph.number_of_edges()  # Archi interni alla comunità
        total_edges = sum(G.degree(n) for n in nodes) / 2  # Archi totali coinvolgenti la comunità
        modularity_local = intra_edges / total_edges if total_edges > 0 else 0

        # 2. Calcolo della closeness inter-comunità
        closeness_sum = 0
        count = 0
        for node in nodes:
            # Calcola la distanza media verso nodi esterni alla comunità
            distances = nx.single_source_shortest_path_length(G, node)
            external_distances = [dist for target, dist in distances.items() if target not in nodes]
            if external_distances:
                closeness_sum += sum(external_distances) / len(external_distances)
                count += 1

        closeness_inter = (closeness_sum / count) if count > 0 else float('inf')

        # Aggiungi risultati per questa comunità
        community_isolation[community] = {
            "modularity_local": modularity_local,
            "closeness_inter": closeness_inter
        }

    return community_isolation


isolated_groups = find_isolated_groups(G)

# Stampa dei risultati
print("Gruppi isolati (ordinati per modularità locale):")
for community, measures in sorted(isolated_groups.items(), key=lambda x: x[1]['modularity_local'], reverse=True):
    print(f"Comunità {community} - Authority {resultsComunities[community]['Authority']}:")
    print(f"  Modularità locale: {measures['modularity_local']:.2f}")
    print(f"  Closeness inter-comunità: {measures['closeness_inter']:.2f}")
    print("-")


Gruppi isolati (ordinati per modularità locale):
Comunità 14 - Authority Alleras:
  Modularità locale: 0.90
  Closeness inter-comunità: 4.90
-
Comunità 6 - Authority Barristan Selmy:
  Modularità locale: 0.88
  Closeness inter-comunità: 3.83
-
Comunità 11 - Authority Jon Snow:
  Modularità locale: 0.86
  Closeness inter-comunità: 3.54
-
Comunità 7 - Authority Balon Greyjoy:
  Modularità locale: 0.82
  Closeness inter-comunità: 3.92
-
Comunità 18 - Authority Waymar Royce:
  Modularità locale: 0.75
  Closeness inter-comunità: 3.91
-
Comunità 15 - Authority Myrcella Baratheon:
  Modularità locale: 0.72
  Closeness inter-comunità: 3.86
-
Comunità 5 - Authority Gerold Hightower:
  Modularità locale: 0.72
  Closeness inter-comunità: 4.23
-
Comunità 16 - Authority Jon Connington:
  Modularità locale: 0.71
  Closeness inter-comunità: 3.53
-
Comunità 1 - Authority Catelyn Stark:
  Modularità locale: 0.71
  Closeness inter-comunità: 3.37
-
Comunità 10 - Authority Tyrion Lannister:
  Modularità l

##**Analisi delle dinamiche interne delle comunità**
Come sono organizzati i legami interni di ogni comunità? Questa analisi è cruciale per individuare alleanze forti, sottogruppi o potenziali divisioni interne.


Metriche usate:
* **Coefficiente di clustering interno**: Misura la densità delle connessioni all'interno della comunità, utile per identificare strutture fortemente coese.
* **Numero di triangoli**: Analizza i gruppi chiusi (triangoli) presenti all'interno della comunità, che rappresentano sottoreti di personaggi con legami forti e stabili.

In [ ]:
def analyze_internal_links(group_df, group_edges):
    # Crea grafo temporaneo
    G_group = nx.Graph()

    # Aggiungi i nodi
    for idx, row in group_df.iterrows():
        G_group.add_node(row['Id'], label=row['Label'], community=row['modularity_class'])

    # Aggiungi gli archi
    for idx, row in group_edges.iterrows():
        source = row['Source']
        target = row['Target']
        G_group.add_edge(source, target)

    # Dizionario per salvare i risultati di ogni comunità
    community_internal_analysis = {}

    # 1. Coefficiente di clustering interno
    clustering_coefficients = nx.clustering(G_group)
    avg_clustering = sum(clustering_coefficients.values()) / len(clustering_coefficients)

    # 2. Numero di triangoli
    triangles = sum(nx.triangles(G_group).values()) // 3  # Ogni triangolo è conteggiato 3 volte

    # Salva i risultati
    community_internal_analysis[community] = {
        "avg_clustering": avg_clustering,
        "num_triangles": triangles
    }

    return community_internal_analysis

resultsInternalAnalysis = {}

for modularity_class in modularity:
    # Filtra il DataFrame per ciascun 'modularity_class'
    group_df = df[df['modularity_class'] == modularity_class]
    group_edges = df_edges[df_edges['Source'].isin(group_df['Id']) & df_edges['Target'].isin(group_df['Id'])]

    # Trova rappresentante con max authority
    rappresentante = group_df.loc[group_df['Authority'].idxmax()]

    # Salva i risultati per ogni classe
    resultsInternalAnalysis[modularity_class] = analyze_internal_links(group_df, group_edges)
    resultsInternalAnalysis[modularity_class]['Authority'] = rappresentante['Label']


print("Analisi dei legami interni di ogni comunità:")
for community in sorted(resultsInternalAnalysis.items(), key=lambda x: x[1].get('avg_clustering', 0), reverse=True):
    print(f"Comunità {community}:")
    avg_clustering = measures.get('avg_clustering', 0)
    num_triangles = measures.get('num_triangles', 0)
    authority = measures.get('Authority', "Nessun nodo")
    print(f"  Coefficiente di clustering medio: {avg_clustering:.2f}")
    print(f"  Numero di triangoli: {num_triangles}")
    print(f"  Rappresentante: {authority}")
    print("-")


Analisi dei legami interni di ogni comunità:
Comunità (0, {13: {'avg_clustering': 0.5598038990636028, 'num_triangles': 308}, 'Authority': 'Arya Stark'}):
  Coefficiente di clustering medio: 0.00
  Numero di triangoli: 0
  Rappresentante: Nessun nodo
-
Comunità (1, {13: {'avg_clustering': 0.5530773314277841, 'num_triangles': 354}, 'Authority': 'Catelyn Stark'}):
  Coefficiente di clustering medio: 0.00
  Numero di triangoli: 0
  Rappresentante: Nessun nodo
-
Comunità (2, {13: {'avg_clustering': 0.4818305080985837, 'num_triangles': 95}, 'Authority': 'Stannis Baratheon'}):
  Coefficiente di clustering medio: 0.00
  Numero di triangoli: 0
  Rappresentante: Nessun nodo
-
Comunità (3, {13: {'avg_clustering': 1.0, 'num_triangles': 1}, 'Authority': 'Unella'}):
  Coefficiente di clustering medio: 0.00
  Numero di triangoli: 0
  Rappresentante: Nessun nodo
-
Comunità (4, {13: {'avg_clustering': 0.0, 'num_triangles': 0}, 'Authority': 'Alys Arryn'}):
  Coefficiente di clustering medio: 0.00
  Nume

## Modifica dataset per ottenere grafi per casate
